## Sliding windows and Classifiers
This is the final procedure of features engineerings. It should be done after the features extraction and after the one-hot-encoding procedure.<br>
The end of the notebook has an example of a some classifers which try to predict single porteins

### Some libraries

In [1]:
#Standard
import sys
import pandas as pd
import numpy as np
import time
import warnings
import matplotlib.pyplot as plt
import pickle
from joblib import dump, load
#Sliding windows
from scipy import signal
#Module
from modules.feature_extraction import *
from modules.models import *
#Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#Metricsc
from sklearn import metrics
print('Done')

Done


### Importing DataFrame 'one-hot-enc'

In [66]:
#Import data
df = pd.read_csv(r'datasets\one-hot-enc.csv')
#Get data ready
LIP = df.LIP
df.drop(['LIP_SCORE', 'LIP'], inplace = True, axis = 1)
#Elimate NaN
df.REL_ASA[df.REL_ASA.isna()] = df.REL_ASA.mean()
df.head()

C:\Users\fgrim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,PDB_ID,CHAIN_ID,RES_ID,REL_ASA,PHI,PSI,NH_O_1_relidx,NH_O_1_energy,O_NH_1_relidx,O_NH_1_energy,...,MC_SC,NO_EDGE_LOC,SC_MC,SC_SC,HBOND,IAC,NO_EDGE_TYPE,PIPISTACK,VDW,CHAIN_LEN
0,1cee,A,1,1.000000,360.0,97.6,0.0,0.0,2.0,-0.3,...,0,1,0,0,0,0,1,0,0,179
1,1cee,A,2,0.348485,-91.3,147.8,48.0,-0.1,50.0,-1.7,...,0,0,0,2,2,0,0,0,1,179
2,1cee,A,3,0.387324,-142.6,136.7,-2.0,-0.3,50.0,-0.2,...,0,1,0,0,0,0,1,0,0,179
3,1cee,A,4,0.005917,-94.3,154.4,48.0,-1.9,50.0,-1.6,...,0,0,0,3,2,0,0,0,3,179
4,1cee,A,5,0.346341,-112.5,70.5,-2.0,-0.2,71.0,-1.6,...,0,0,1,0,1,0,0,0,1,179


### Apply sliding windows

Use odd numbers for windows or some extra NaN could be generated

In [79]:
df_slided.columns

Index(['PDB_ID', 'CHAIN_ID', 'RES_ID', 'REL_ASA', 'PHI', 'PSI',
       'NH_O_1_relidx', 'NH_O_1_energy', 'O_NH_1_relidx', 'O_NH_1_energy',
       'NH_O_2_relidx', 'H_O_2_energy', 'O_NH_2_relidx', 'O_NH_2_energy',
       'INTRA_CONTACTS', 'INTER_CONTACTS', 'ALA', 'ARG', 'ASN', 'ASP', 'CYS',
       'GDP', 'GLN', 'GLU', 'GLY', 'GTP', 'HIS', 'HYP', 'ILE', 'LEU', 'LYS',
       'MET', 'MSE', 'PHE', 'PRO', 'PTR', 'SEP', 'SER', 'THR', 'TPO', 'TRP',
       'TYR', 'VAL', '10_ELIX', 'ALPHA_ELIX', 'BEND', 'ISOLATED_BETA_BRIGE',
       'NO_STRUCT', 'PI_ELIX', 'STRAND', 'TURN', 'ZERO', 'LIG_MC', 'LIG_SC',
       'MC_MC', 'MC_SC', 'NO_EDGE_LOC', 'SC_MC', 'SC_SC', 'HBOND', 'IAC',
       'NO_EDGE_TYPE', 'PIPISTACK', 'VDW', 'CHAIN_LEN'],
      dtype='object')

### Model - Leave-One-Out of Protein
Here we run a cycle that for every protein of the dataset it use that protein as test set and the rest as training. <br>
We print Accuracy, Precision, Recall and Confusion Matrix

#### Build Classifier

In [82]:
clf = RandomForestClassifier(n_estimators=120)

#### Validation phase - LOOCV
Very long procedure for some classifiers. Use verbose to see results for each protein (advised if testing the model for the first time)

In [31]:
results = loo_cv(data = df_slided, clf=clf, 
                 target = LIP,
                 bad_condition=(0.75, 0.75),
                 ign_warnings = True, 
                 get_times = True, verbose = True)

Ieration number: 1
1cee
Accuracy: 0.966
Balanced Accuracy: 0.955
Precision: 0.898
Recall: 0.936
F1 score: 0.917
[[186   5]
 [  3  44]]
______________________________________________________
Ieration number: 2
1dev
Accuracy: 0.991
Balanced Accuracy: 0.976
Precision: 1.0
Recall: 0.951
F1 score: 0.975
[[194   0]
 [  2  39]]
______________________________________________________
Ieration number: 3
1dow
Accuracy: 0.996
Balanced Accuracy: 0.984
Precision: 1.0
Recall: 0.968
F1 score: 0.984
[[206   0]
 [  1  30]]
______________________________________________________
Ieration number: 4
1fqj
Accuracy: 0.972
Balanced Accuracy: 0.985
Precision: 0.714
Recall: 1.0
F1 score: 0.833
[[321  10]
 [  0  25]]
______________________________________________________
Ieration number: 5
1g3j
Accuracy: 1.0
Balanced Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0
[[522   0]
 [  0  41]]
______________________________________________________
Ieration number: 6
1hrt
Accuracy: 0.708
Balanced Accuracy: 0.604
P

Ieration number: 46
1tce
Accuracy: 0.975
Balanced Accuracy: 0.987
Precision: 0.667
Recall: 1.0
F1 score: 0.8
[[112   3]
 [  0   6]]
______________________________________________________
Ieration number: 47
1r1r
Accuracy: 0.997
Balanced Accuracy: 0.999
Precision: 0.875
Recall: 1.0
F1 score: 0.933
[[734   2]
 [  0  14]]
______________________________________________________
Ieration number: 48
1mxl
Accuracy: 0.991
Balanced Accuracy: 0.995
Precision: 0.923
Recall: 1.0
F1 score: 0.96
[[93  1]
 [ 0 12]]
______________________________________________________
Ieration number: 49
2fym
Accuracy: 1.0
Balanced Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0
[[858   0]
 [  0  15]]
______________________________________________________
Ieration number: 50
1iwq
Accuracy: 1.0
Balanced Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0
[[139   0]
 [  0  18]]
______________________________________________________
Ieration number: 51
1fv1
Accuracy: 0.9
Balanced Accuracy: 0.8
Precision: 0.882

#### Get some results of the model

In [13]:
print('The avarage balanced accuracy is {}\n'.format(results[1]))
print('The avarage f1 score is {}\n'.format(results[3]))
print('The proteins which gave bad results are:\n{}\n'.format(results[4]))
print('Number of proteins which gave bad results: {}\n'.format(len(results[4])))

The avarage balanced accuracy is 0.9308333333333335

The avarage f1 score is 0.8927222222222222

The proteins which gave bad results are:
[('1hrt', 0.601, 0.796), ('1i7w', 1.0, 0.0), ('1jsu', 0.797, 0.745), ('1kil', 0.44, 0.584), ('1rf8', 0.5, 0.0), ('1sc5', 0.766, 0.622), ('1sqq', 0.986, 0.719), ('1ymh', 0.6, 0.774), ('2a6q', 0.643, 0.913), ('1j2x', 0.559, 0.333)]

Number of proteins which gave bad results: 10



### Example of testing for 1cee

In [89]:
X_train = np.array(df_slided.iloc[238:, 3:])
y_train = np.array(LIP[238:])
X_test = np.array(df_slided.iloc[:238, 3:])
y_test = np.array(LIP[:238])

clf.fit(X_train, y_train)
metrics.balanced_accuracy_score(y_test, clf.predict(X_test))

0.957613902194497

In [81]:
pred = []
with open(r'C:\\Users\\fgrim\\Downloads\\predict_1cee (1).txt') as table:
    next(table)
    for line in table:
            pred.append(int(line[-2]))
            
metrics.balanced_accuracy_score(pred, y_test)

0.9232085346215781

### Features Selection

In [14]:
if type(clf) == RandomForestClassifier:    
    feat_imp = []
    for i,j in zip(clf.feature_importances_, df_slided.columns[3:]):
        feat_imp.append((i, j))

sorted(feat_imp, reverse=True)

[(0.5930578964049581, 'CHAIN_LEN'),
 (0.04689653799002892, 'O_NH_2_relidx'),
 (0.04649243405499608, 'REL_ASA'),
 (0.04185741616133256, 'NH_O_2_relidx'),
 (0.030807925554730827, 'NH_O_1_relidx'),
 (0.02959554793750593, 'O_NH_1_relidx'),
 (0.021604513022796746, 'NH_O_1_energy'),
 (0.02053518054896099, 'PSI'),
 (0.02022562775972197, 'O_NH_1_energy'),
 (0.020092291679689785, 'PHI'),
 (0.018001063205129404, 'O_NH_2_energy'),
 (0.01430946312822856, 'H_O_2_energy'),
 (0.010321958217923656, 'NO_STRUCT'),
 (0.0047538314596514974, 'LYS'),
 (0.004572599555106869, 'LEU'),
 (0.004433149728987181, 'GLY'),
 (0.004315807076432857, 'SER'),
 (0.004097523829422096, 'GLU'),
 (0.004065458506264279, 'ALPHA_ELIX'),
 (0.003816636122903383, 'ASP'),
 (0.0037134918277674732, 'BEND'),
 (0.0033637837749413693, 'VAL'),
 (0.0033438385677021184, 'ALA'),
 (0.0032626705749551014, 'STRAND'),
 (0.003224954462051298, 'THR'),
 (0.0032191550284069313, 'TURN'),
 (0.0031282445165079376, 'ARG'),
 (0.0030792206423954985, 'GLN')

### New DataSet with less features

In [81]:
low_feat = list(df_slided.columns[47:64])
df_new = df_slided.copy()
df_new.drop(low_feat, axis = 1, inplace = True)
df_new.head()

,PDB_ID,CHAIN_ID,RES_ID,REL_ASA,PHI,PSI,NH_O_1_relidx,NH_O_1_energy,O_NH_1_relidx,O_NH_1_energy,...,THR,TPO,TRP,TYR,VAL,10_ELIX,ALPHA_ELIX,BEND,ISOLATED_BETA_BRIGE,CHAIN_LEN
0,1cee,A,1,0.474244,83.082501,92.296821,19.408981,-0.040435,20.884355,-0.787401,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,132.045992
1,1cee,A,2,0.396646,13.519922,96.636711,15.595646,-0.093986,27.179865,-0.667755,...,0.202177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,132.045992
2,1cee,A,3,0.200760,-85.057363,106.664522,18.742314,-0.504354,36.884355,-0.733850,...,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,132.045992
3,1cee,A,4,0.150303,-83.008657,93.357718,15.191292,-0.734422,41.130070,-0.897252,...,0.202177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,132.045992
4,1cee,A,5,0.116643,-74.013146,76.510780,18.742314,-0.814721,43.884355,-1.463347,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,132.045992


### Re-test the model

In [36]:
results = loo_cv(data = df_new, clf=clf, 
                 target = LIP,
                 bad_condition=(0.75, 0.75),
                 ign_warnings = True, 
                 get_times = True, verbose = True)

NameError: name 'df_new' is not defined

In [17]:
print('The avarage balanced accuracy is {}\n'.format(results[1]))
print('The avarage f1 score is {}\n'.format(results[3]))
print('The proteins which gave bad results are:\n{}\n'.format(results[4]))
print('Number of proteins which gave bad results: {}\n'.format(len(results[4])))

The avarage balanced accuracy is 0.9303888888888889

The avarage f1 score is 0.8936944444444445

The proteins which gave bad results are:
[('1hrt', 0.526, 0.76), ('1i7w', 1.0, 0.0), ('1jsu', 0.79, 0.734), ('1kil', 0.409, 0.581), ('1rf8', 0.5, 0.0), ('1sc5', 0.768, 0.63), ('1sqq', 0.986, 0.719), ('1ymh', 0.533, 0.768), ('1j2x', 0.531, 0.319)]

Number of proteins which gave bad results: 9



### Save MODEL

In [19]:
clf = RandomForestClassifier(n_estimators=120)
X, y = np.array(df.iloc[:, ])
dump(clf, 'decision_tree.joblib') 

['decision_tree.joblib']

### Import model

In [39]:
clf = load('decision_tree.joblib')

In [40]:
y_pred = clf.predict(df_slided.iloc[0:300, 3:])
y_test = LIP[0:300]
metrics.balanced_accuracy_score(y_pred, y_test)

0.9430034962192049